# 第一章：設定你的 Go 環境

歡迎來到 Go 語言學習之旅！在這個章節中，我們將學習如何設定 Go 開發環境，並熟悉基本的 Go 工具。

## 學習目標
- 安裝和設定 Go 開發環境
- 了解 Go 工作區和模組系統
- 掌握基本的 Go 命令
- 學習程式碼格式化和檢查工具
- 選擇適合的開發工具

## 1. 安裝 Go 工具

### 檢查 Go 是否已安裝
首先，讓我們檢查系統是否已經安裝了 Go：

In [1]:
package main

import (
    "fmt"
    "runtime"
)

func main() {
    fmt.Printf("Go 版本: %s\n", runtime.Version())
    fmt.Printf("作業系統: %s\n", runtime.GOOS)
    fmt.Printf("架構: %s\n", runtime.GOARCH)
}

Go 版本: go1.24.4
作業系統: darwin
架構: arm64


### 安裝指南

如果你還沒有安裝 Go，可以按照以下步驟：

#### macOS
```bash
# 使用 Homebrew（推薦）
brew install go

# 或下載官方安裝包
# 從 https://golang.org/dl/ 下載 .pkg 檔案
```

#### Linux
```bash
# Ubuntu/Debian
sudo apt update && sudo apt install golang-go

# 或下載官方二進制檔案
wget https://golang.org/dl/go1.21.0.linux-amd64.tar.gz
sudo tar -C /usr/local -xzf go1.21.0.linux-amd64.tar.gz
```

#### Windows
從 [golang.org/dl/](https://golang.org/dl/) 下載並執行 `.msi` 安裝程式

## 2. Go 工作區與環境變數

讓我們檢查 Go 的環境設定：

In [1]:
package main

import (
    "fmt"
    "os"
    "runtime"
)

func main() {
    fmt.Println("=== Go 環境資訊 ===")
    
    // GOROOT - Go 安裝路徑
    fmt.Printf("GOROOT: %s\n", runtime.GOROOT())
    
    // GOPATH - Go 工作區路徑（舊版本使用）
    gopath := os.Getenv("GOPATH")
    if gopath != "" {
        fmt.Printf("GOPATH: %s\n", gopath)
    } else {
        fmt.Println("GOPATH: 未設定（使用 Go Modules）")
    }
    
    // 其他重要環境變數
    fmt.Printf("GOOS: %s\n", runtime.GOOS)
    fmt.Printf("GOARCH: %s\n", runtime.GOARCH)
    fmt.Printf("NumCPU: %d\n", runtime.NumCPU())
}

=== Go 環境資訊 ===
GOROOT: /opt/homebrew/Cellar/go/1.24.4/libexec
GOPATH: /Users/hankhsieh/go
GOOS: darwin
GOARCH: arm64
NumCPU: 8


### Go Modules vs GOPATH：依賴管理的演進

#### 傳統的 GOPATH 模式（Go 1.11 之前）

在 Go Modules 出現之前，Go 使用 **GOPATH** 來管理程式碼和依賴：

**GOPATH 的問題：**
- 所有 Go 程式碼必須放在 `$GOPATH/src` 目錄下
- 無法在同一台機器上使用同一個套件的不同版本
- 依賴管理困難，容易出現「依賴地獄」
- 專案必須放在特定的目錄結構中

```
GOPATH/
├── bin/          # 編譯後的執行檔
├── pkg/          # 編譯後的套件檔案
└── src/          # 所有原始碼必須放這裡
    ├── github.com/
    │   └── user/project/
    └── example.com/
        └── another/project/
```

#### 現代的 Go Modules（Go 1.11+）

**Go Modules 的優勢：**
- 可以在任何目錄建立 Go 專案
- 支援語義化版本控制（Semantic Versioning）
- 每個專案可以使用不同版本的依賴
- 自動下載和管理依賴
- 更好的建置重現性

#### 建立新的 Go Module 專案
```bash
mkdir my-go-project
cd my-go-project
go mod init my-go-project
```

這會建立一個 `go.mod` 檔案來管理模組資訊：

```
module my-go-project

go 1.21

require (
    github.com/gin-gonic/gin v1.9.1
    github.com/stretchr/testify v1.8.4
)
```

#### go get 如何自動更新 go.mod 檔案

**重要概念：`go get` 會自動更新 `go.mod` 和 `go.sum` 檔案**

當你執行 `go get` 命令時，Go 會：
1. 下載指定的套件到 `$GOMODCACHE`
2. **自動更新 `go.mod` 檔案**，添加新的依賴
3. **自動更新 `go.sum` 檔案**，記錄校驗和
4. 解析並下載所有間接依賴

**實際操作流程：**

```bash
# 1. 初始的 go.mod
module my-project
go 1.21

# 2. 執行 go get 命令
go get github.com/gin-gonic/gin@v1.9.1

# 3. go.mod 會自動變成：
module my-project
go 1.21

require github.com/gin-gonic/gin v1.9.1

require (
    github.com/bytedance/sonic v1.9.1 // indirect
    github.com/chenzhuoyu/base64x v0.0.0-20221115062448-fe3a3abad311 // indirect
    // ... 其他間接依賴會自動添加
)

# 4. 同時會建立 go.sum 檔案記錄校驗和
```

#### 常用的 go get 命令變化

```bash
# 添加最新版本依賴
go get github.com/gin-gonic/gin

# 添加特定版本的依賴
go get github.com/gin-gonic/gin@v1.9.1

# 升級到最新版本
go get -u github.com/gin-gonic/gin

# 升級所有依賴到最新版本
go get -u ./...

# 移除依賴（會從 go.mod 中移除）
go get github.com/gin-gonic/gin@none

# 添加特定 commit 的版本
go get github.com/gin-gonic/gin@abc1234

# 添加特定分支的版本
go get github.com/gin-gonic/gin@master
```

#### go.mod 和 go.sum 的關係

- **go.mod**: 記錄模組的依賴關係和版本
- **go.sum**: 記錄每個依賴的校驗和，確保安全性和一致性

**go.sum 範例內容：**
```
github.com/gin-gonic/gin v1.9.1 h1:4idEAncQnU5cB7BeOkPtxjfCSye0AAm1R0RVIqJ+Jmg=
github.com/gin-gonic/gin v1.9.1/go.mod h1:hPrL7YrpYKXt5YId3A/Tnip5kqbEAP+KLuI3SUcPTeU=
```

#### 其他重要的 Go Modules 命令

```bash
# 初始化新模組
go mod init <module-name>

# 移除未使用的依賴，添加缺少的依賴
go mod tidy

# 下載依賴到本地快取（不修改 go.mod）
go mod download

# 查看模組依賴圖
go mod graph

# 查看為什麼需要某個依賴
go mod why <package>

# 驗證依賴的完整性
go mod verify

# 查看可用的版本
go list -m -versions github.com/gin-gonic/gin
```

#### 實際示範 go get 更新 go.mod

讓我們寫個程式來檢查 go.mod 檔案的變化：

In [2]:
package main

import (
    "fmt"
    "io/ioutil"
    "os"
    "path/filepath"
    "strings"
)

func main() {
    fmt.Println("=== go get 如何更新 go.mod 檔案 ===")
    
    // 檢查當前目錄的 go.mod 檔案
    pwd, _ := os.Getwd()
    goModPath := filepath.Join(pwd, "go.mod")
    goSumPath := filepath.Join(pwd, "go.sum")
    
    fmt.Printf("當前目錄: %s\n\n", pwd)
    
    // 讀取 go.mod 檔案
    if content, err := ioutil.ReadFile(goModPath); err == nil {
        fmt.Println("📄 當前 go.mod 內容:")
        fmt.Println(strings.Repeat("-", 40))
        fmt.Println(string(content))
        fmt.Println(strings.Repeat("-", 40))
    } else {
        fmt.Println("❌ 沒有找到 go.mod 檔案")
        fmt.Println("💡 請先執行: go mod init <module-name>")
    }
    
    // 檢查 go.sum 檔案
    if _, err := os.Stat(goSumPath); err == nil {
        fmt.Println("\n✅ 找到 go.sum 檔案 - 記錄依賴的校驗和")
    } else {
        fmt.Println("\n❌ 沒有找到 go.sum 檔案")
        fmt.Println("💡 當你執行 go get 添加依賴時會自動建立")
    }
    
    fmt.Println("\n🔍 重要提醒:")
    fmt.Println("- go get 會自動修改 go.mod 檔案")
    fmt.Println("- go get 會自動建立/更新 go.sum 檔案")
    fmt.Println("- 間接依賴會標記為 // indirect")
    fmt.Println("- 使用 go mod tidy 可以清理未使用的依賴")
    
    fmt.Println("\n📝 試試看這些命令:")
    fmt.Println("go get github.com/gin-gonic/gin@v1.9.1")
    fmt.Println("go mod tidy")
    fmt.Println("然後再次執行這個程式看看 go.mod 的變化！")
}

=== go get 如何更新 go.mod 檔案 ===
當前目錄: /Users/hankhsieh/Workspace/github/learning-go/ch1_setup_env

❌ 沒有找到 go.mod 檔案
💡 請先執行: go mod init <module-name>

❌ 沒有找到 go.sum 檔案
💡 當你執行 go get 添加依賴時會自動建立

🔍 重要提醒:
- go get 會自動修改 go.mod 檔案
- go get 會自動建立/更新 go.sum 檔案
- 間接依賴會標記為 // indirect
- 使用 go mod tidy 可以清理未使用的依賴

📝 試試看這些命令:
go get github.com/gin-gonic/gin@v1.9.1
go mod tidy
然後再次執行這個程式看看 go.mod 的變化！


#### Go Modules 實際操作示範

讓我們建立一個簡單的程式來示範 Go Modules 的功能：

In [3]:
package main

import (
    "fmt"
    "os"
    "path/filepath"
)

func main() {
    fmt.Println("=== Go Modules 資訊 ===")
    
    // 取得當前工作目錄
    pwd, _ := os.Getwd()
    fmt.Printf("當前目錄: %s\n", pwd)
    
    // 檢查是否有 go.mod 檔案
    goModPath := filepath.Join(pwd, "go.mod")
    if _, err := os.Stat(goModPath); err == nil {
        fmt.Println("✅ 找到 go.mod 檔案 - 這是一個 Go Module 專案")
    } else {
        fmt.Println("❌ 沒有找到 go.mod 檔案 - 請執行 'go mod init <module-name>'")
    }
    
    // 檢查 GOPATH（可能未設定）
    gopath := os.Getenv("GOPATH")
    if gopath != "" {
        fmt.Printf("GOPATH: %s\n", gopath)
        fmt.Println("💡 注意: 使用 Go Modules 時不需要依賴 GOPATH")
    } else {
        fmt.Println("GOPATH: 未設定（推薦使用 Go Modules）")
    }
}

=== Go Modules 資訊 ===
當前目錄: /Users/hankhsieh/Workspace/github/learning-go/ch1_setup_env
❌ 沒有找到 go.mod 檔案 - 請執行 'go mod init <module-name>'
GOPATH: /Users/hankhsieh/go
💡 注意: 使用 Go Modules 時不需要依賴 GOPATH


#### 常用的 Go Modules 命令

```bash
# 初始化新模組
go mod init <module-name>

# 添加依賴（會自動更新 go.mod 和 go.sum）
go get github.com/gin-gonic/gin

# 添加特定版本的依賴
go get github.com/gin-gonic/gin@v1.9.1

# 移除未使用的依賴，添加缺少的依賴
go mod tidy

# 下載依賴到本地快取
go mod download

# 查看模組依賴圖
go mod graph

# 查看為什麼需要某個依賴
go mod why <package>

# 驗證依賴的完整性
go mod verify
```

#### go.mod 檔案結構說明

```go
module example.com/myproject    // 模組名稱

go 1.21                        // Go 版本要求

require (                      // 直接依賴
    github.com/gin-gonic/gin v1.9.1
    github.com/stretchr/testify v1.8.4
)

require (                      // 間接依賴（自動管理）
    github.com/bytedance/sonic v1.9.1 // indirect
    github.com/chenzhuoyu/base64x v0.0.0-20221115062448-fe3a3abad311 // indirect
)

replace (                      // 替換依賴（用於本地開發）
    github.com/old/package => github.com/new/package v1.0.0
)

exclude (                      // 排除特定版本
    github.com/problematic/package v1.2.3
)
```

#### Go Modules 依賴存放位置

**重要澄清：Go Modules 的依賴存放在哪裡？**

Go Modules 的依賴**不是**存放在傳統的 GOPATH 中，而是有自己的快取機制：

```bash
# Go Modules 快取位置（預設）
~/go/pkg/mod/          # 模組快取目錄
~/go/pkg/sumdb/        # 校驗和資料庫快取
```

**關鍵差異：**
- **GOPATH 模式**: 依賴存放在 `$GOPATH/src/`，每個套件只能有一個版本
- **Go Modules 模式**: 依賴存放在 `$GOMODCACHE`（預設 `~/go/pkg/mod/`），支援多版本並存

**實際範例：**
```
~/go/pkg/mod/
├── github.com/gin-gonic/gin@v1.9.1/
├── github.com/gin-gonic/gin@v1.8.2/    # 可以同時存在多個版本
├── github.com/stretchr/testify@v1.8.4/
└── cache/                               # 下載快取
```

**環境變數控制：**
```bash
# 可以自訂模組快取位置
export GOMODCACHE=/path/to/custom/cache

# 檢查目前的模組快取位置
go env GOMODCACHE
```

#### Go Modules 的優勢總結

| 特性 | GOPATH 模式 | Go Modules 模式 |
|------|-------------|------------------|
| 專案位置 | 必須在 `$GOPATH/src` | 任何目錄都可以 |
| 依賴存放 | `$GOPATH/src/` | `$GOMODCACHE` (預設 `~/go/pkg/mod/`) |
| 版本管理 | 無版本控制 | 語義化版本控制 |
| 依賴隔離 | 全域共享 | 每個專案獨立 |
| 建置重現性 | 困難 | 透過 go.sum 保證 |
| 依賴下載 | 手動管理 | 自動下載和快取 |
| 多版本支援 | 不支援 | 完全支援 |

#### 檢查 Go Modules 快取位置

讓我們寫個程式來檢查實際的模組快取位置：

In [ ]:
package main

import (
    "fmt"
    "os"
    "os/exec"
    "path/filepath"
    "strings"
)

func main() {
    fmt.Println("=== Go Modules 快取位置檢查 ===")
    
    // 使用 go env 命令檢查模組快取位置
    cmd := exec.Command("go", "env", "GOMODCACHE")
    output, err := cmd.Output()
    if err == nil {
        gomodcache := strings.TrimSpace(string(output))
        fmt.Printf("GOMODCACHE: %s\n", gomodcache)
        
        // 檢查目錄是否存在
        if _, err := os.Stat(gomodcache); err == nil {
            fmt.Println("✅ 模組快取目錄存在")
        } else {
            fmt.Println("❌ 模組快取目錄不存在")
        }
    }
    
    // 檢查傳統 GOPATH（如果有設定）
    gopath := os.Getenv("GOPATH")
    if gopath != "" {
        fmt.Printf("\nGOPATH: %s\n", gopath)
        fmt.Println("💡 注意: Go Modules 不使用 GOPATH/src，而是使用 GOMODCACHE")
    } else {
        // 預設的 GOPATH
        homeDir, _ := os.UserHomeDir()
        defaultGoPath := filepath.Join(homeDir, "go")
        fmt.Printf("\n預設 GOPATH: %s\n", defaultGoPath)
        fmt.Println("💡 Go Modules 使用 ~/go/pkg/mod/ 而不是 ~/go/src/")
    }
    
    fmt.Println("\n🔍 總結:")
    fmt.Println("- GOPATH 模式: 依賴存放在 $GOPATH/src/")
    fmt.Println("- Go Modules: 依賴存放在 $GOMODCACHE (通常是 ~/go/pkg/mod/)")
    fmt.Println("- 兩者是完全不同的存放機制！")
}

## 3. 你的第一個 Go 程式

讓我們從經典的 "Hello, World!" 開始：

In [ ]:
package main

import "fmt"

func main() {
    fmt.Println("Hello, World!")
    fmt.Println("你好，Go 語言！")
    fmt.Println("歡迎來到 Go 的世界 🚀")
}

## 4. 基本 Go 命令

### go run vs go build

- `go run`: 直接執行程式碼，不產生執行檔
- `go build`: 編譯程式碼並產生執行檔

讓我們示範一個簡單的程式來說明差異：

In [4]:
package main

import (
    "fmt"
    "os"
)

func main() {
    fmt.Println("程式名稱:", os.Args[0])
    fmt.Println("這個程式可以用以下方式執行:")
    fmt.Println("1. go run program.go")
    fmt.Println("2. go build program.go && ./program")
    
    if len(os.Args) > 1 {
        fmt.Println("命令列參數:", os.Args[1:])
    }
}

程式名稱: /var/folders/yk/8pm6f7xd5p175t_q_0wl13sc0000gn/T/gonb_61b338af/gonb_61b338af
這個程式可以用以下方式執行:
1. go run program.go
2. go build program.go && ./program


### 其他重要的 Go 命令

```bash
go fmt      # 格式化程式碼
go vet      # 檢查程式碼問題
go test     # 執行測試
go mod tidy # 整理模組依賴
go get      # 下載依賴套件
```

## 5. 程式碼格式化

Go 有官方的程式碼格式化工具 `go fmt`。讓我們看看格式化前後的差異：

In [5]:
package main

import "fmt"

// 這個函數示範了 Go 的標準格式
func demonstrateFormatting() {
    // Go 會自動格式化縮排、空格等
    numbers := []int{1, 2, 3, 4, 5}
    
    for i, num := range numbers {
        fmt.Printf("索引 %d: 值 %d\n", i, num)
    }
    
    // 條件語句的格式
    if len(numbers) > 0 {
        fmt.Println("陣列不是空的")
    } else {
        fmt.Println("陣列是空的")
    }
}

func main() {
    fmt.Println("=== 程式碼格式化示範 ===")
    demonstrateFormatting()
    
    fmt.Println("\n提示: 使用 'go fmt' 命令來自動格式化程式碼")
}

=== 程式碼格式化示範 ===
索引 0: 值 1
索引 1: 值 2
索引 2: 值 3
索引 3: 值 4
索引 4: 值 5
陣列不是空的

提示: 使用 'go fmt' 命令來自動格式化程式碼
